###Make graphs with weighted edges

In [1]:
import pandas as pd
import pickle
import numpy as np
import networkx as nx

In [2]:
votes = pickle.load( open("process_votes.p", "rb" ) )

In [3]:
votes['Borneo']

1,Richard,Kelly,Rudy,Susan,Sean,Colleen,Gervase,Jenna,Greg,Gretchen,Joel,Dirk,Ramona,Stacey,B.B.,Sonja
1,,,,,,,,,,,,,,,,
Richard,None,5,9,8,5,0,0,1,0,0,0,1,0,0,0,0
Kelly,None,None,6,6,2,1,0,1,0,0,0,0,0,2,0,1
Rudy,None,None,None,9,6,0,0,1,0,0,0,2,0,0,0,0
Susan,None,None,None,None,6,0,0,1,0,0,0,2,0,0,0,0
Sean,None,None,None,None,None,0,0,1,0,0,0,2,0,0,0,0
Colleen,None,None,None,None,None,None,4,4,2,3,2,0,1,0,0,0
Gervase,None,None,None,None,None,None,None,2,1,1,2,0,2,0,0,0
Jenna,None,None,None,None,None,None,None,None,1,3,2,0,1,0,0,0
Greg,None,None,None,None,None,None,None,None,None,1,0,0,0,0,1,0


In [4]:
# Make network graphs
def make_graphs(votes):
    
    # Create adjacency matrix
    votes = votes.fillna(0) #convert None to zeroes
    votes_matrix = votes.as_matrix() #creates numpy array
    votes_matrix = np.matrix(votes_matrix) #creates matrix

    # Make graph
    G = nx.from_numpy_matrix(votes_matrix)
    
    # Rename nodes
    names = list(votes.columns.values)
    nodeints = list(range(len(names)))
    rename = {}
    for i in nodeints:
        rename[i] = names[i]
    G = nx.relabel_nodes(G, rename)
        
    return G

In [5]:
graphs = {}
for i in votes.keys():
    graphs[i] = make_graphs(votes[i])

In [6]:
graphs

{u'Africa': <networkx.classes.graph.Graph at 0xad13198>,
 u'All-Stars': <networkx.classes.graph.Graph at 0xab74400>,
 u'Blood_vs._Water': <networkx.classes.graph.Graph at 0xabf4588>,
 u'Borneo': <networkx.classes.graph.Graph at 0xad13f60>,
 u'Cagayan': <networkx.classes.graph.Graph at 0xab74518>,
 u'Cambodia': <networkx.classes.graph.Graph at 0xabf4f60>,
 u'Caramoan': <networkx.classes.graph.Graph at 0xab74470>,
 u'China': <networkx.classes.graph.Graph at 0xab74780>,
 u'Cook_Islands': <networkx.classes.graph.Graph at 0xab74198>,
 u'Fiji': <networkx.classes.graph.Graph at 0xab742e8>,
 u'Gabon': <networkx.classes.graph.Graph at 0xab74438>,
 u'Guatemala': <networkx.classes.graph.Graph at 0xabf4d68>,
 u'Heroes_vs._Villains': <networkx.classes.graph.Graph at 0xab74208>,
 u'Heroes_vs_Villains': <networkx.classes.graph.Graph at 0xabf4198>,
 u'Marquesas': <networkx.classes.graph.Graph at 0xabf4390>,
 u'Micronesia': <networkx.classes.graph.Graph at 0xab743c8>,
 u'Nicaragua': <networkx.classes.g

In [7]:
graphs['Borneo'].edges(data=True)

[(u'Gervase', u'Ramona', {'weight': 2}),
 (u'Gervase', u'Joel', {'weight': 2}),
 (u'Gervase', u'Greg', {'weight': 1}),
 (u'Gervase', u'Gretchen', {'weight': 1}),
 (u'Gervase', u'Colleen', {'weight': 4}),
 (u'Gervase', u'Jenna', {'weight': 2}),
 (u'Gretchen', u'Ramona', {'weight': 1}),
 (u'Gretchen', u'Joel', {'weight': 2}),
 (u'Gretchen', u'Greg', {'weight': 1}),
 (u'Gretchen', u'Colleen', {'weight': 3}),
 (u'Gretchen', u'Jenna', {'weight': 3}),
 (u'B.B.', u'Greg', {'weight': 1}),
 (u'Ramona', u'Colleen', {'weight': 1}),
 (u'Ramona', u'Jenna', {'weight': 1}),
 (u'Ramona', u'Joel', {'weight': 1}),
 (u'Richard', u'Susan', {'weight': 8}),
 (u'Richard', u'Dirk', {'weight': 1}),
 (u'Richard', u'Sean', {'weight': 5}),
 (u'Richard', u'Kelly', {'weight': 5}),
 (u'Richard', u'Rudy', {'weight': 9}),
 (u'Richard', u'Jenna', {'weight': 1}),
 (u'Susan', u'Dirk', {'weight': 2}),
 (u'Susan', u'Sean', {'weight': 6}),
 (u'Susan', u'Kelly', {'weight': 6}),
 (u'Susan', u'Rudy', {'weight': 9}),
 (u'Susan'

####Save info

In [8]:
pickle.dump(graphs, open( "make_graphs.p", "wb" ) )